<br>

# Introdução


In [ ]:
#!pip3 install open_geodata

In [ ]:
import re

import folium
import geopandas as gpd
import pandas as pd
import seaborn as sns
from folium import plugins
from open_geodata import lyr

from sp_mpsp_divadmin.paths import (
    output_path_geo,
    output_path_gpkg,
    output_path_maps,
)

<br>

## Parâmetros


In [ ]:
FILLOPACITY = 0.5
SIMPLIFY = 0.0005

In [ ]:
color_polygon_pjreg = {
    'Promotoria de Justiça Regional do Meio Ambiente do Aguapeí/Peixe': '#a6cee3',  #
    'Promotoria de Justiça Regional do Meio Ambiente do Alto Mogi/Médio Mogi Superior': '#1f78b4',  #
    'Promotoria de Justiça Regional do Meio Ambiente do Alto Paranapanema': '#b2df8a',  #
    'Promotoria de Justiça Regional do Meio Ambiente do Alto Tietê/Grande': '#33a02c', #
    'Promotoria de Justiça Regional do Meio Ambiente do Baixo Pardo/Grande': '#fb9a99',
    'Promotoria de Justiça Regional do Meio Ambiente do Baixo Tietê': '#e31a1c',
    'Promotoria de Justiça Regional do Meio Ambiente do Cabeceiras': '#fdbf6f',
    'Promotoria de Justiça Regional do Meio Ambiente do Cotia/Pirapora': '#ff7f00',
    'Promotoria de Justiça Regional do Meio Ambiente do Litoral Norte': '#cab2d6',
    'Promotoria de Justiça Regional do Meio Ambiente do Médio Mogi Inferior': '#6a3d9a',
    'Promotoria de Justiça Regional do Meio Ambiente do Médio Paranapanema': '#ffff99',
    'Promotoria de Justiça Regional do Meio Ambiente do PCJ Campinas-Atibaia': '#e31a1c',
    'Promotoria de Justiça Regional do Meio Ambiente do PCJ Piracicaba': '#a6cee3',
    'Promotoria de Justiça Regional do Meio Ambiente do Paraíba do Sul': '#1f78b4',
    'Promotoria de Justiça Regional do Meio Ambiente do Paraíba do Sul/ Baixada': '#b2df8a',
    'Promotoria de Justiça Regional do Meio Ambiente do Pardo': '#33a02c',
    'Promotoria de Justiça Regional do Meio Ambiente do Pontal do Paranapanema': '#fb9a99',
    'Promotoria de Justiça Regional do Meio Ambiente do Sapucaí/Grande': '#e31a1c',
    'Promotoria de Justiça Regional do Meio Ambiente do São José dos Dourados': '#fdbf6f',
    'Promotoria de Justiça Regional do Meio Ambiente do Tietê/Batalha': '#ff7f00',
    'Promotoria de Justiça Regional do Meio Ambiente do Tietê/Jacaré': '#cab2d6',
    'Promotoria de Justiça Regional do Meio Ambiente do Tietê/Sorocaba': '#6a3d9a',
    'Promotoria de Justiça Regional do Meio Ambiente do Turvo/Grande': '#ffff99',
    'Promotoria de Justiça Regional do Meio Ambiente do Vale do Ribeira': '#b15928',
}

color_polygon_gaema = {
    'Núcleo Baixada Santista': '#b2df8a',
    'Núcleo Cabeceiras': '#fdbf6f',
    'Núcleo Litoral Norte': '#cab2d6',
    'Núcleo Médio Paranapanema': '#ffff99',
    'Núcleo PCJ-Atibaia': '#fb9a99',
    'Núcleo PCJ-Campinas': '#e31a1c',
    'Núcleo PCJ-Piracicaba': '#a6cee3',
    'Núcleo Pardo': '#33a02c',
    'Núcleo Pontal do Paranapanema': '#fb9a99',
    'Núcleo Tietê/Sorocaba': '#6a3d9a',
    'Núcleo Vale do Paraíba': '#1f78b4',
    'Núcleo Vale do Ribeira': '#b15928',
}

color_polygon_rp = {
    'Aguapeí/Peixe': '#a6cee3',
    'Alto Mogi': '#1f78b4',
    'Alto Paranapanema': '#b2df8a',
    'Alto Tietê': '#33a02c',
    'Baixo Pardo/Grande': '#fb9a99',
    'Baixo Tietê': '#e31a1c',
    'Cotia/Pirapora': '#ff7f00',
    'Mantiqueira': '#ff7f00',
    'Médio Mogi Inferior': '#6a3d9a',
    'Médio Mogi Superior': '#6a3d9a',
    'Sapucaí/Grande': '#e31a1c',
    'São José dos Dourados': '#fdbf6f',
    'Tietê/Batalha': '#ff7f00',
    'Tietê/Jacaré': '#cab2d6',
    'Turvo/Grande': '#ffff99',
}

<br>

## _Layer_: GAEMA


In [ ]:
def add_lyr_gaema():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_mpsp_gaema.gpkg')
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)
    gdf = gdf[gdf['id_gaema'].notnull()]

    # Column with category
    col_categories = 'gaema_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    #print(color_polygon)
    # color_polygon['1º Batalhão'] = '#ff4d4d'
    # color_polygon['2º Batalhão'] = '#4da6ff'
    # color_polygon['3º Batalhão'] = '#00b300'
    # color_polygon['4º Batalhão'] = '#ffcc99'
    # color_polygon1 = color_polygon

    # Popup
    gdf['popup_gaema'] = gdf.apply(popup_gaema, axis=1)

    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='GAEMA',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon_gaema[x['properties'][col_categories]],
            'color': color_polygon_gaema[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': FILLOPACITY,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'gaema_nome'],
            aliases=['Munícipio', 'GAEMA'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_gaema'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        zoom_on_click=True,
        embed=False,
        show=False,
    )
    return lyr


# Add Field
def popup_gaema(row):
    # tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    # fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')

    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        ''
        if pd.isnull(row['municipio_nome'])
        else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['gaema_nome']) else '{}'.format(row['gaema_nome']),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## _Layer_: RP


In [ ]:
def add_lyr_redeprotetiva():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_mpsp_rp.gpkg')
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)
    gdf = gdf[gdf['rp_nome'].notnull()]

    # Column with category
    col_categories = 'rp_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    print(color_polygon)
    # color_polygon['1º Batalhão'] = '#ff4d4d'
    # color_polygon['2º Batalhão'] = '#4da6ff'
    # color_polygon['3º Batalhão'] = '#00b300'
    # color_polygon['4º Batalhão'] = '#ffcc99'
    # color_polygon1 = color_polygon

    # Popup
    gdf['popup_redeprotetiva'] = gdf.apply(popup_redeprotetiva, axis=1)

    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Rede Protetiva',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon_rp[x['properties'][col_categories]],
            'color': color_polygon_rp[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': FILLOPACITY,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'rp_nome'],
            aliases=['Munícipio', 'Rede Protetiva'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_redeprotetiva'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        zoom_on_click=True,
        embed=False,
        show=False,
    )
    return lyr


# Add Field
def popup_redeprotetiva(row):
    # tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    # fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')

    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        ''
        if pd.isnull(row['municipio_nome'])
        else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['rp_nome']) else '{}'.format(row['rp_nome']),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## _Layer_: AR


In [ ]:
def add_lyr_ar():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_mpsp_ar.gpkg')
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)
    gdf = gdf[gdf['id_ar'].notnull()]

    # Column with category
    col_categories = 'ar_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    color_polygon['Bauru'] = '#ff4d4d'
    # color_polygon['2º Batalhão'] = '#4da6ff'
    # color_polygon['3º Batalhão'] = '#00b300'
    # color_polygon['4º Batalhão'] = '#ffcc99'
    # color_polygon1 = color_polygon
    # print(color_polygon)

    # Popup
    gdf['popup_ar'] = gdf.apply(popup_ar, axis=1)

    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Área Regional',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': FILLOPACITY,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'ar_nome'],
            aliases=['Munícipio', 'AR'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_ar'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        zoom_on_click=True,
        embed=False,
        show=False,
    )
    return lyr


# Add Field
def popup_ar(row):
    # tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    # fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')

    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        ''
        if pd.isnull(row['municipio_nome'])
        else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['ar_nome']) else '{}'.format(row['ar_nome']),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## _Layer_: Macrorregião


In [ ]:
def add_lyr_macroregioes():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_mpsp_macrorregioes.gpkg')
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)
    gdf = gdf[gdf['id_macrorregiao'].notnull()]

    # Column with category
    col_categories = 'macrorregiao_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    # color_polygon['1º Batalhão'] = '#ff4d4d'
    # color_polygon['2º Batalhão'] = '#4da6ff'
    # color_polygon['3º Batalhão'] = '#00b300'
    # color_polygon['4º Batalhão'] = '#ffcc99'
    # color_polygon1 = color_polygon

    # Popup
    gdf['popup_macro'] = gdf.apply(popup_macro, axis=1)

    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Macrorregiões',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': FILLOPACITY,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'macrorregiao_nome'],
            aliases=['Munícipio', 'Macrorregião'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_macro'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        zoom_on_click=True,
        embed=False,
        show=False,
    )
    return lyr


# Add Field
def popup_macro(row):
    # tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    # fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')

    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        ''
        if pd.isnull(row['municipio_nome'])
        else '{}'.format(row['municipio_nome']),
        ''
        if pd.isnull(row['macrorregiao_nome'])
        else '{}'.format(row['macrorregiao_nome']),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## _Layer_: PJ Regional


In [ ]:
def add_lyr_pjreg():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_mpsp_pjreg.gpkg')
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)
    gdf = gdf[gdf['id_pjreg'].notnull()]

    # Column with category
    col_categories = 'pjreg_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    print(color_polygon)
    # color_polygon['1º Batalhão'] = '#ff4d4d'
    # color_polygon['2º Batalhão'] = '#4da6ff'
    # color_polygon['3º Batalhão'] = '#00b300'
    # color_polygon['4º Batalhão'] = '#ffcc99'
    # color_polygon1 = color_polygon

    # Popup
    gdf['popup_macro'] = gdf.apply(popup_pjreg, axis=1)

    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='Promotoria de Justiça Regional do Meio Ambiente',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon_pjreg[x['properties'][col_categories]],
            'color': color_polygon_pjreg[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': FILLOPACITY,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'pjreg_nome'],
            aliases=['Munícipio', 'PJ Regional'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_macro'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        zoom_on_click=True,
        embed=False,
        show=True,
    )
    return lyr


# Add Field
def popup_pjreg(row):
    # tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    # fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')

    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        ''
        if pd.isnull(row['municipio_nome'])
        else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['pjreg_nome']) else '{}'.format(row['pjreg_nome']),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## _Layer_: GAECO


In [ ]:
def add_lyr_gaeco():
    # Input
    gdf = gpd.read_file(output_path_gpkg / 'sp_mpsp_gaeco.gpkg')
    gdf = gdf.to_crs(epsg=4326)
    gdf['geometry'] = gdf.simplify(SIMPLIFY)
    gdf = gdf[gdf['id_gaeco_mun'].notnull()]

    # Column with category
    col_categories = 'gaeco_nome'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    #print(color_polygon)
    # color_polygon['1º Batalhão'] = '#ff4d4d'
    # color_polygon['2º Batalhão'] = '#4da6ff'
    # color_polygon['3º Batalhão'] = '#00b300'
    # color_polygon['4º Batalhão'] = '#ffcc99'
    # color_polygon1 = color_polygon

    # Popup
    gdf['popup_macro'] = gdf.apply(popup_gaeco, axis=1)

    # Geo
    lyr = folium.GeoJson(
        gdf,
        name='GAECO',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': FILLOPACITY,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'gaeco_nome'],
            aliases=['Munícipio', 'GAECO'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup_macro'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,
        ),
        zoom_on_click=True,
        embed=False,
        show=False,
    )
    return lyr


# Add Field
def popup_gaeco(row):
    # tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    # fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')

    html = """
    <div>
    <p><b>{}</b> pertence ao:
    <h5><b>{}</b></h5></p>
    </div>
    """.format(
        ''
        if pd.isnull(row['municipio_nome'])
        else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['gaeco_nome']) else '{}'.format(row['gaeco_nome']),
    )

    html = html.replace('\n', '')
    html = re.sub('\s\s+', ' ', html)  # Remove Espaços no meio
    html = html.strip()
    return html

<br>

# Map


In [ ]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]

    # Zoom
    min_zoom = 6
    max_zoom = 11

    # Create Map
    m = folium.Map(
        # zoom_start=6,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        # zoom_delta=0.1,
        min_lat=bounds[0][0] * (101 / 100),
        min_lon=bounds[0][1] * (101 / 100),
        max_lat=bounds[1][0] * (99 / 100),
        max_lon=bounds[1][1] * (99 / 100),
        tiles=None,
    )

    # Add Layers
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))
    m.add_child(add_lyr_gaema())
    m.add_child(add_lyr_redeprotetiva())
    m.add_child(add_lyr_gaeco())
    m.add_child(add_lyr_ar())
    m.add_child(add_lyr_macroregioes())
    m.add_child(add_lyr_pjreg())

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
# Map without Bounds
m = get_map(output_path_gpkg / 'sp_mpsp_ar.gpkg')

# Results
m.save(output_path_maps / 'mpsp_map.html')
m